### Install and Import Dependencies

In [1]:
!pip install opencv-python

In [2]:
pip install mediapipe-model-maker==0.2.1.3

  Obtaining dependency information for mediapipe-model-maker==0.2.1.3 from https://files.pythonhosted.org/packages/9a/f4/26b03c5b5cacd9bcbaf06a906915f56d6b7cd7276c28c246db4a42c1f35b/mediapipe_model_maker-0.2.1.3-py3-none-any.whl.metadata
  Using cached mediapipe_model_maker-0.2.1.3-py3-none-any.whl.metadata (1.6 kB)
  Obtaining dependency information for tensorflow-addons from https://files.pythonhosted.org/packages/40/5b/570cfbe49634b934c7379a4c2c57aed7cc747dade3879426e036cebc3fe3/tensorflow_addons-0.23.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached tensorflow_addons-0.23.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (1.7 kB)
  Obtaining dependency information for tensorflow-datasets from https://files.pythonhosted.org/packages/fe/18/4865973f5469cfe33bbe1cfc2f1918335eb44f4cc3d316c1bce22c1af2bc/tensorflow_datasets-4.9.4-py3-none-any.whl.metadata
  Using cached tensorflow_datasets-4.9.4-py3-none-any.whl.metadata (9.2 kB)
  Obtaining dependency information for tensorflow-hub 

In [3]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

objc[2955]: Class CaptureDelegate is implemented in both /Users/edwinyue/anaconda3/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x17950a620) and /Users/edwinyue/anaconda3/lib/python3.11/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x117f54860). One of the two will be used. Which one is undefined.
objc[2955]: Class CVWindow is implemented in both /Users/edwinyue/anaconda3/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x17950a670) and /Users/edwinyue/anaconda3/lib/python3.11/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x110950a68). One of the two will be used. Which one is undefined.
objc[2955]: Class CVView is implemented in both /Users/edwinyue/anaconda3/lib/python3.11/site-packages/cv2/cv2.abi3.so (0x17950a698) and /Users/edwinyue/anaconda3/lib/python3.11/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x110950a90). One of the two will be used. Which one is undefined.
objc[2955]: Class CVSlider is implemented in both /Users/edwin

### Make Detections

### Determining Joints

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [24]:
#landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

0.9983878135681152

In [25]:
#landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

x: 0.99030375
y: 1.2366229
z: -0.33999118
visibility: 0.20503038

In [26]:
#landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

x: 0.9904549
y: 1.6641017
z: -0.7625021
visibility: 0.025868502

In [38]:
#landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]

x: 0.27493525
y: 0.94814736
z: -0.5150083
visibility: 0.696514

In [39]:
#landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]

x: 0.16135724
y: 1.4517808
z: -1.2381706
visibility: 0.4696596

# 3. Calculate Angles

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

### Double Arm Curl Counter

In [ ]:
cap = cv2.VideoCapture(0)

# Curl counter variables
Lcounter = 0 
Rcounter = 0
Lstage = None
Rstage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            Lshoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            Lelbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            Lwrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            # Get coordinates
            Rshoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
            Relbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
            Rwrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
            
            # Calculate angle
            Langle = calculate_angle(Lshoulder, Lelbow, Lwrist)
             # Calculate angle
            Rangle = calculate_angle(Rshoulder, Relbow, Rwrist)
            
            
            # Visualize angle
            cv2.putText(image, str(Langle), 
                           tuple(np.multiply(Lelbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                )
            # Visualize angle
            cv2.putText(image, str(Rangle), 
                           tuple(np.multiply(Relbow, [640, 480]).astype(int)), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                       )
            
              

            # Curl counter logic
            if Langle < 45:
                Lstage = "down"
            elif Langle > 55 and Lstage =='down':
                Lstage="up"
                Lcounter +=1
                print(Lcounter)

            if Rangle < 45:
                Rstage = "down"
            elif Rangle > 55 and Rstage =='down':
                Rstage="up"
                Rcounter +=1
                print(Rcounter)
                
         
                       
        except:
            pass
        
        # Render curl counter
        # Setup status box
        cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(Lcounter), 
                    (10,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        cv2.rectangle(image, (2000,0), (1600,73), (245,117,16), -1)
        cv2.putText(image, 'REPS', (15,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(Rcounter), 
                    (1800,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, Lstage, 
                    (60,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
        cv2.putText(image, 'STAGE', (65,12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, Rstage, 
                    (1620,60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)    
        
        # Render detections
        if (Lstage == "up"):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=3, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(230,0,110), thickness=2, circle_radius=2) 
                                )  
        if (Rstage == "up"):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=3, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(230,110,0), thickness=2, circle_radius=2) 
                                )  
        if (Rstage == "down" and Lstage == "down"):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=3, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(230,0,0), thickness=2, circle_radius=2) 
                                )  
        if (Rstage == "up" and Lstage == "up"):
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                mp_drawing.DrawingSpec(color=(0,0,0), thickness=3, circle_radius=2), 
                                mp_drawing.DrawingSpec(color=(230,110,110), thickness=2, circle_radius=2) 
                                )          
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
    for i in range (1,5):
        cv2.waitKey(1)